# IESB - Miner II - Aula 05 - Random Forest

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

In [ ]:
# Vamos visualizar os dados
df_all.info()

In [ ]:
# Vamos aumentar o número de colunas para o info mostrar
df_all.info(max_cols=145)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

### Vamos investigar o valor 'yes' na coluna dependency

In [ ]:
# Vamos obter o idhogar de alguns casos
df_all[df_all['dependency'] == 'yes']['idhogar'].head(20)

In [ ]:
# Vamos investigar o idhogar 2b58d945f pegando as variáveis de cálculo da coluna dependency
df_all[df_all['idhogar'] == '2b58d945f'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

In [ ]:
# Vamos investigar o idhogar d6dae86b7 pegando as variáveis de cálculo da coluna dependency
df_all[df_all['idhogar'] == 'd6dae86b7'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

In [ ]:
# Vamos investigar o idhogar 652a7ffa0 pegando as variáveis de cálculo da coluna dependency
df_all[df_all['idhogar'] == '652a7ffa0'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

In [ ]:
# Vamos obter o idhogar de alguns casos
df_all[df_all['dependency'] == 'no']['idhogar'].head(20)

In [ ]:
# Vamos investigar o idhogar 3e16fab89 pegando as variáveis de cálculo da coluna dependency
df_all[df_all['idhogar'] == '3e16fab89'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

In [ ]:
# Vamos investigar o idhogar 3e16fab89 pegando as variáveis de cálculo da coluna dependency
df_all[df_all['idhogar'] == 'bcc196e5a'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

In [ ]:
# Analisando os dados da coluna edjefa
df_all['edjefa'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefe
df_all['edjefe'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# nas colunas edjefa e edjefe
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# na coluna dependency
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Visualizando do comando info
df_all.info(max_cols=145)

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum()

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum().sort_values(ascending=False)

In [ ]:
 # Verificando os valores de aluguel (v2a1) para os chefes/as de familia (parentesco1 = 1)
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Qual a cara dos dados de v18q
df_all['v18q'].value_counts()

In [ ]:
# Prenchendo com -1 os valores nulos de v2a1
df_all['v2a1'].fillna(-1, inplace=True)

In [ ]:
# Prenchendo com 0 os valores nulos de v18q1
df_all['v18q1'].fillna(0, inplace=True)

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum().sort_values()

In [ ]:
# Vamos obter o idhogar de alguns casos onde o meanaeduc é vazio
df_all[df_all['meaneduc'].isnull()]['idhogar'].head(20)

In [ ]:
# Investigando a casa com ID = e197a50ea
df_all[df_all['idhogar'] == 'e197a50ea'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'edjefe', 'edjefa', 'Target']]

In [ ]:
# Investigando a casa com ID = a874b7ce7
df_all[df_all['idhogar'] == 'a874b7ce7'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'edjefe', 'edjefa', 'Target']]

In [ ]:
# Prenchendo com -1 os valores nulos de SQBmeaned, meaneduc e rez_esc
df_all['SQBmeaned'].fillna(-1, inplace=True)
df_all['meaneduc'].fillna(-1, inplace=True)
df_all['rez_esc'].fillna(-1, inplace=True)

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Instanciando o random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

In [ ]:
# Treinando o modelo
rf.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# Vamos verificar as previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Vamos verificar a distribuição da variável Target na base de treino
train['Target'].value_counts(normalize=True)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(15, 20))

# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

## Vamos usar a variável idhogar para treinar o modelo

In [ ]:
# Transformando idhogar em variável numérica
df_all['idhogar'] = df_all['idhogar'].astype('category').cat.codes

df_all.info()

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Instanciando o random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf2 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

In [ ]:
# Treinando o modelo
rf2.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf2.predict(test[feats]).astype(int)

In [ ]:
# Vamos verificar as previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)